In [1]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Strategy")
from cn_general_tool import *
from cn_stock_strategy_factor import *

In [2]:
sd, ed = get_sd_ed('2019-12-31', '2024-11-25')

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    dividend_yield_ratio AS factor
FROM cn_stock_prefactors
QUALIFY c_pct_rank(total_market_cap) > 0.20
AND c_pct_rank(pe_ttm) < 0.40
AND pe_ttm > 0 
AND ps_ttm < 2.5
"""

In [4]:
sql_trade = cn_stock_strategy_factor_trade_sql(sql_factor, number_security=10, position_allocation="equal", rebalance_period="month", stock_pool="000300.SH")

In [5]:
strategy_df = get_dai_df(sql_trade, sd, ed)
strategy_df

,date,instrument,score,score_rank,position
0,2019-12-31,600188.SH,0.145833,1.0,0.1
1,2019-12-31,600019.SH,0.087108,2.0,0.1
2,2019-12-31,600028.SH,0.074364,3.0,0.1
3,2019-12-31,000898.SZ,0.065672,4.0,0.1
4,2019-12-31,601006.SH,0.058465,5.0,0.1
...,...,...,...,...,...
585,2024-10-31,601006.SH,0.078521,6.0,0.1
586,2024-10-31,600039.SH,0.072418,7.0,0.1
587,2024-10-31,601229.SH,0.059355,8.0,0.1
588,2024-10-31,601916.SH,0.057746,9.0,0.1


In [6]:
df1 = cn_stock_strategy_factor_backtest(strategy_df, capital_base=1000000)

[2024-12-01 14:05:53] [info     ] bigtrader.v34 开始运行 ..
[2024-12-01 14:05:54] [info     ] 2019-12-31, 2024-10-31, , , instruments=67
[2024-12-01 14:05:55] [info     ] bigtrader module V2.2.0
[2024-12-01 14:05:55] [info     ] bigtrader engine v1.10.10 2024-11-26
[2024-12-01 14:06:10] [info     ] backtest done, raw_perf_ds:dai.DataSource("_fe2e08b40161460594d9d138badb9ecb")


[2024-12-01 14:06:15] [info     ] bigtrader.v34 运行完成 [21.960s].


In [7]:
df2 = cn_stock_strategy_factor_market_neutral_backtest(strategy_df, capital_base=1000000)

[2024-12-01 14:06:16] [info     ] bigtrader.v34 开始运行 ..
[2024-12-01 14:06:16] [info     ] 2019-12-31, 2024-10-31, , future, instruments=67
[2024-12-01 14:06:16] [info     ] bigtrader module V2.2.0
[2024-12-01 14:06:16] [info     ] bigtrader engine v1.10.10 2024-11-26


INFO:MAIN:======== bigtrader pid:1511 version 1.10.10 2024-11-26 ========
INFO:MAIN:bigtrader run_mode:BACKTEST, handle_bar_mode:0, frequency:1d, exchange_mode:BQ2
INFO:MAIN:> process add account:BACKTEST,1,bktfut
INFO:ACCT[bktfut]:AccountEngine: self_calc:1, validate_self_trading:0, validate_cash:0, validate_position:1, enable_auto_planed_order:1
INFO:MAIN:login_account(bktfut)
INFO:MAIN:> add_strategy setting:{'strategy_name': 'strategy', 'account_id': 'bktfut'}
INFO:MAIN:init all strategy account_id:...


INFO:MAIN:stop all strategy account_id:...


[2024-12-01 14:06:20] [info     ] backtest done, raw_perf_ds:dai.DataSource("_6eb99a076d9342c6b2ac5377076a8406")


[2024-12-01 14:06:22] [info     ] bigtrader.v34 运行完成 [6.072s].


In [15]:
df1_new = df1[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_stock'}, inplace=False).reset_index(drop=True)
df2_new = df2[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_index'}, inplace=False).reset_index(drop=True)

In [23]:
df_merge = pd.merge(df1_new, df2_new, on='date', how='inner')
df_merge['portfolio_value'] = df_merge['portfolio_value_stock'] + df_merge['portfolio_value_index']
df_merge['portfolio_value']       = df_merge['portfolio_value']       / df_merge['portfolio_value'].iloc[0]
df_merge['portfolio_value_stock'] = df_merge['portfolio_value_stock'] / df_merge['portfolio_value_stock'].iloc[0]
df_merge['portfolio_value_index'] = df_merge['portfolio_value_index'] / df_merge['portfolio_value_index'].iloc[0]

df_merge

,date,portfolio_value_stock,portfolio_value_index,portfolio_value
0,2019-12-31,1.000000,1.000000,1.000000
1,2020-01-02,0.999701,0.999971,0.999836
2,2020-01-03,1.000354,1.002551,1.001453
3,2020-01-06,0.994588,1.008311,1.001450
4,2020-01-07,1.000669,0.999371,1.000020
...,...,...,...,...
1165,2024-10-25,1.563460,1.346984,1.455222
1166,2024-10-28,1.581578,1.342724,1.462151
1167,2024-10-29,1.558292,1.352384,1.455338
1168,2024-10-30,1.534677,1.365584,1.450130


In [24]:
from bigcharts import Line
Line(df_merge)